# Obtain and Clean LODES Data

> The LEHD Origin-Destination Employment Statistics (LODES) datasets are released both as
part of the OnTheMap application and in raw form as a set of comma separated variable (CSV)
text files. This document describes the structure of those raw files and provides basic information
for users who want to perform analytical work on the data outside of the OnTheMap application." (U.S. Census, 2021)

U.S. Census Bureau. (2021). LEHD Origin-Destination Employment Statistics Data (2002-2018) [computer file]. Washington, DC: U.S. Census Bureau, Longitudinal-Employer Household Dynamics Program [distributor], accessed on {CURRENT DATE} at https://lehd.ces.census.gov/data/#lodes. LODES 7.5 [version]

The LODES Data provides emplyoment characterstics and origin-destination data
    
    1. Read in LODES data
    2. Select Work Area Charactersitics in Study Area
    3. Select Origin-Destination data in Study Area

## Description of Program
- program:    LODES_1av4_CleanLODESdata
- task:       Obtain and read in LODES data
- See github commits for description of program updates
- Current Version:    2022-01-07
- Version 4 description - try to run with fewer charactersitics and allow for lower fitness
-     Need to get a result in less time
- project:    Interdependent Networked Community Resilience Modeling Environment (IN-CORE) Subtask 5.2 - Social Institutions
- funding:	  NIST Financial Assistance Award Numbers: 70NANB15H044 and 70NANB20H008 
- author:     Nathanael Rosenheim

- Suggested Citation:
Rosenheim, N. (2021) “Obtain, Clean, and LODES Jobs Data". 
Archived on Github and ICPSR.

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import os # For saving output to path

In [ ]:
# Display versions being used - important information for replication
import sys
print("Python Version     ", sys.version)
print("numpy version:     ", np.__version__)
print("pandas version:    ", pd.__version__)

In [ ]:
os.getcwd()

In [ ]:
# Store Program Name for output files to have the same name
programname = "LODES_1av4_CleanLODESdata"
# Save Outputfolder - due to long folder name paths output saved to folder with shorter name
# files from this program will be saved with the program name - this helps to follow the overall workflow
outputfolder = "lodes_workflow_outputv4"
# Make directory to save output
if not os.path.exists(outputfolder):
    os.mkdir(outputfolder)

### Setup notebook enviroment to access Cloned Github Package
This notebook uses packages that are in developement. The packages are available at:

https://github.com/npr99/Labor_Market_Allocation

To replicate this notebook Clone the Github Package to a folder that is a sibling of this notebook.

To access the sibling package you will need to append the parent directory ('..') to the system path list.

In [ ]:
# to access new package that is in a sibling folder - the system path list needs to inlcude the parent folder (..)
# append the path of the directory that includes the github repository.
sys.path.append("..\\github_com\\npr99\\Labor_Market_Allocation")

# Setup access to IN-CORE
https://incore.ncsa.illinois.edu/

In [ ]:
#from pyincore import IncoreClient, Dataset, FragilityService, MappingSet, DataService
#from pyincore_viz.geoutil import GeoUtil as viz

### IN-CORE addons
This program uses coded that is being developed as potential add ons to pyincore. These functions are in a folder called pyincore_addons - this folder is located in the same directory as this notebook.
The add on functions are organized to mirror the folder sturcture of https://github.com/IN-CORE/pyincore

Each add on function attempts to follow the structure of existing pyincore functions and includes some help information.

In [ ]:
# To reload submodules need to use this magic command to set autoreload on
%load_ext autoreload
%autoreload 2
# open, read, and execute python program with reusable commands
# function that loops through lodes data structure
import pyincoredata_addons.SourceData.lehd_ces_census_gov.lodes_fullloop as lodes
import pyincoredata_addons.SourceData.lehd_ces_census_gov.lodes_mcmcsa_loops as mcmc
import pyincoredata_addons.SourceData.lehd_ces_census_gov.lodes_mcmcsa_util as mcmc_util
import pyincoredata_addons.SourceData.lehd_ces_census_gov.lodes_datautil as data_util


# since the geoutil is under construction it might need to be reloaded
#from importlib import reload 
#lodes = reload(lodes) # with auto reload on this command is not needed

# Print list of add on functions
##from inspect import getmembers, isfunction
##print(getmembers(lodes,isfunction))
##print(getmembers(mcmc,isfunction))

## Read In LODES data for a County

In [ ]:
# Setup list and dictionaries to loop through
# countylist can only have 1 county at this time
countylist = {'37155' : 'Robeson County, NC'}
# note 2010 is the earliest year with Federal Jobtypes 
# to include years before 2010 requires modifiying the program
#years = ['2010','2011','2012','2013','2014','2015','2016','2017','2018']
#years = ['2012','2013','2014','2015','2016']
years = ['2010']

In [ ]:
stacked_df = lodes.obtain_lodes_county_loop(countylist = countylist, 
                        years = years,
                        outputfoldername = outputfolder)

In [ ]:
lodes.explore_jobcounts(stacked_df)

In [ ]:
focus_jobtype = 'JT11'
seed = 133234

stacked_jobtype_df = mcmc.split_stack_df_byjobtype(stacked_df)
for county in countylist:
    countyfips = county
    joblist_df, mcmcsa_filepath = \
        mcmc.outer_mcmc_sa_input("2010", years,
                    focus_jobtype,
                    stacked_jobtype_df,
                    outputfolder,
                    countyfips,
                    seed
                    )

In [ ]:
    
# ## Run Markov-Chain-Monte Carlo Simmulated Anneal
"""
The MCMC SA process will take the combined WAC-OD-RAC joblist and 
futher reduce the number of possible jobs. 
This process will randomly select jobs and compare the job 
characteristics to the known WAC file job characteristics.
"""

random_accept_threshold = 0.1
# How much to reduce error by
start_reduction_threshold = 0.6
max_reduction_threshold = 0.9
seed = 133234

joblist_mcmcsa_df = mcmc.outer_mcmc_sa_loop(
                years = years,
                focus_jobtype = focus_jobtype,
                stacked_jobtype_df = stacked_jobtype_df,
                joblist_df = joblist_df,
                seed = seed,
                random_accept_threshold = random_accept_threshold,
                mcmcsa_filepath = mcmcsa_filepath,
                start_reduction_threshold = start_reduction_threshold,
                num_procs=8
                )

In [ ]:
compare_C000, total_fitness, combined_fitness = \
    mcmc.compare_expected_possible(joblist_mcmcsa_df, stacked_jobtype_df)

In [ ]:
compare_C000

In [ ]:
total_fitness

In [ ]:
combined_fitness